# Unsupervised Learning
1. Clustering
    * Group similar instances together into cluster.
        * ex: data analysis, customer segmentation, recommender system, search engine, image segmentation, semi-supervised learning, dimensionality reduction
2. Anomaly detection
    * To learn what "normal" data looks like, and then use that to detect abnormal instances.
        * ex: defective items on the production line, new trend in a time-series
3. Density estimation
    * Estimating the probability density function (PDF) - commonly used for anomaly detection
